<details><summary> </summary>

# Skip notebook test

</details>

In [ ]:
import cudf
from cuopt import routing
import numpy as np
import os
from cuopt.routing import utils
from scipy.spatial import distance

# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

While other notebooks such as [cvrptw_service_team_routing.ipynb](cvrptw_service_team_routing.ipynb) focus on the cuOpt API and high level problem modeling, here we focus on performance.

cuOpt offers a unique benefit over other solver_settingss, specifically, time to solution.  In addition to achieving world class accuracy, cuOpt also produces these solutions in a time frame that allows for re-optimization in dynamic environments and rapid iteration over possible problem configurations.

Here we are demonstrating this performance on a large popular academic [dataset by Gehing & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/).  These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems.  Each problem instance has an associated best known solution, the one we will measure against is shown below

In [ ]:
homberger_1000_file = 'notebook_utils/data/C1_10_1.TXT'

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Problem Data
The data for this problem instance are provided via text file.  cuOpt has a utility function available specifically for the Gehring & Homberger benchmark which converts the problem into the components required by cuOpt.

In [ ]:
orders, vehicle_capacity, n_vehicles = utils.create_from_file(homberger_1000_file)

print("Number of locations          : ", orders["demand"].shape[0]-1)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

### Cost Matrix

In [ ]:
coords = list(zip(orders['xcord'].to_arrow().to_pylist(),
                  orders['ycord'].to_arrow().to_pylist()))

cost_matrix = cudf.DataFrame(distance.cdist(coords, coords, 'euclidean')).astype(np.float32)
print(f"Shape of cost matrix: {cost_matrix.shape}")

### cuOpt DataModel View

Setup the routing.DataModel.

In [ ]:
n_locations = len(cost_matrix)

data_model = routing.DataModel(n_locations, n_vehicles)
data_model.add_cost_matrix(cost_matrix)

capacity = cudf.Series([vehicle_capacity] * n_vehicles)
data_model.add_capacity_dimension("demand", orders['demand'], capacity)

data_model.set_order_time_windows(orders['earliest_time'], orders['latest_time'])
data_model.set_order_service_times(orders['service_time'])

### CuOpt SolverSettings

Set up routing.SolverSettings.

In [ ]:
solver_settings = routing.SolverSettings()

# set number of climbers that will try to search for an optimal routes in parallel
solver_settings.set_number_of_climbers(2048)

### Solution

Here we will examine the quality of the solution we increase the time budget provided to cuOpt

In [ ]:
def solve_problem(data_model, solver_settings, problem_size):
    routing_solution = routing.Solve(data_model, solver_settings)
    if routing_solution.get_status() == 0:
        print("Cost for the routing in time: ", routing_solution.final_cost)
        print("Vehicle count to complete routing: ", routing_solution.vehicle_count)
        utils.show_vehicle_routes(routing_solution.route, ["Depot"]+[str(i) for i in range(1, problem_size+1)])
    else:
        print("NVIDIA cuOpt Failed to find a solution with status : ", routing_solution.get_status())
        
    return(routing_solution.vehicle_count, routing_solution.final_cost)

In [ ]:
def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

**1 Second Time Limit**

In [ ]:
solver_settings.set_time_limit(1)
vehicles, cost = solve_problem(data_model, solver_settings, len(cost_matrix))

In [ ]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

**10 Second Time Limit**

In [ ]:
solver_settings.set_time_limit(10)
vehicles, cost = solve_problem(data_model, solver_settings, len(cost_matrix))

In [ ]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

**20 Second Time Limit**

In [ ]:
solver_settings.set_time_limit(20)
vehicles, cost = solve_problem(data_model, solver_settings, len(cost_matrix))

In [ ]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

_____

#### SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

#### SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the "Software"),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the
Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
DEALINGS IN THE SOFTWARE.

---